<a href="https://colab.research.google.com/github/SSR-ds/Apache-Spark-and-Big-Data/blob/main/Actions_and_Transformations_on_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing java 8 as spark is written in scala and requires jvm
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [3]:
!ls

sample_data  spark-2.4.7-bin-hadoop2.7	spark-2.4.7-bin-hadoop2.7.tgz


In [4]:
!pip install -U pyarrow

     |████████████████████████████████| 17.7MB 1.3MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [5]:
import os 

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7" 

In [6]:
import findspark  

In [7]:
findspark.init()

In [8]:
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.executor.memory", "4g")
spark.conf.set("spark.driver.memory", "4g")
spark.conf.set("spark.memory.fraction", "0.9")

In [10]:
import sys, tempfile, urllib
import urllib.request
from pyspark.sql.functions import *

In [11]:
BASE_DIR = '/tmp'
CORONA_FILE = os.path.join(BASE_DIR, 'coronavirus_dataset.csv')

In [12]:
CV_data = urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/corona_dataset.csv',CORONA_FILE)

In [13]:
!ls /tmp

blockmgr-f049a57c-13c9-4463-aff1-6e2ce669b532
coronavirus_dataset.csv
dap_multiplexer.943e7428053b.root.log.INFO.20210113-155253.49
dap_multiplexer.INFO
debugger_1kipmsgz44
hsperfdata_root
initgoogle_syslog_dir.0
spark-3391e47e-ef36-46eb-b4e6-503786434ae0
spark-c7b88000-e2b9-4c02-a5c4-65bdaeb24d2b


In [14]:
BASE_DIR = '/tmp'
TWITTER_FILE = os.path.join(BASE_DIR, 'twitter_dataset.csv')

In [15]:
TWITTER_data = urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/tweets.csv',TWITTER_FILE)

In [16]:
!ls /tmp

blockmgr-f049a57c-13c9-4463-aff1-6e2ce669b532
coronavirus_dataset.csv
dap_multiplexer.943e7428053b.root.log.INFO.20210113-155253.49
dap_multiplexer.INFO
debugger_1kipmsgz44
hsperfdata_root
initgoogle_syslog_dir.0
spark-3391e47e-ef36-46eb-b4e6-503786434ae0
spark-c7b88000-e2b9-4c02-a5c4-65bdaeb24d2b
twitter_dataset.csv


In [17]:
CV_data = spark.read.option("inferSchema", "true").csv("/tmp/coronavirus_dataset.csv",header=True)

In [18]:
TWITTER_data = spark.read.option("inferSchema", "true").csv("/tmp/twitter_dataset.csv",header=True)

In [19]:
CV_data.show()

+---+----------------+--------------------+--------+---------+-------------------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|               Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+-------------------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22 00:00:00|      2.0|  0.0|      0.0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22 00:00:00|      2.0|  0.0|      0.0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22 00:00:00|      0.0|  0.0|      0.0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22 00:00:00|      0.0|  0.0|      0.0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|   

In [20]:
TWITTER_data.show()

+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0| geo|                text|                user|            location|            entities|           sentiment|             country|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|null|What is God sayin...|          petodinice|               Lagos|[('about #', 'CAR...|{'neg': 0.0, 'neu...|             Nigeria|
|  1|null|"BREAKING: ""this...| but i took the t...|                -… "|     JerryfranksonJF|      Abuja, Nigeria|"[(""Arsenal's Mi...|
|  2|null| #Coronavirus tes...|              cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|                 USA|
|  3|null| Get ready for ma...|        InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|                 USA|
|  4|null| The #coronavirus...|          

In [21]:
CV_data.count()

27075

In [22]:
TWITTER_data.count()  

1000

In [23]:
TWITTER_data.filter("country=='Australia'").show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+---------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|  country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+---------+
|  7|null| 📽️Friends, I wi...|        LorseaR|New South Wales, ...|[('Twitter', 'GPE...|{'neg': 0.123, 'n...|Australia|
| 77|null| There is no obje...|     chaser1066|    Sydney Australia|['', ('FamiliesFi...|{'neg': 0.224, 'n...|Australia|
|107|null|how long before p...|       Mr_Ant87|Canberra, ACT, Au...|                ['']|{'neg': 0.254, 'n...|Australia|
|144|null| The general solu...|  AlexDevantier|           Melbourne|['', ('EXAC', 'OR...|{'neg': 0.0, 'neu...|Australia|
|150|null| President  has t...|    Doyle_Media|     Sydney • 1.0 AU|[('Coronavirus', ...|{'neg': 0.0, 'neu...|Australia|
|276|null| Deserted #Rome: ...|  

In [24]:
TWITTER_data.filter("country=='Australia' and location like 'Syd%'").show()

+---+----+--------------------+------------+--------------------+--------------------+--------------------+---------+
|_c0| geo|                text|        user|            location|            entities|           sentiment|  country|
+---+----+--------------------+------------+--------------------+--------------------+--------------------+---------+
| 77|null| There is no obje...|  chaser1066|    Sydney Australia|['', ('FamiliesFi...|{'neg': 0.224, 'n...|Australia|
|150|null| President  has t...| Doyle_Media|     Sydney • 1.0 AU|[('Coronavirus', ...|{'neg': 0.0, 'neu...|Australia|
|276|null| Deserted #Rome: ...|ColettaFrank|Sydney NSW Australia|[('Rome', 'GPE'),...|{'neg': 0.0, 'neu...|Australia|
|324|null| President  annou...| Doyle_Media|     Sydney • 1.0 AU|[('Europe', 'LOC'...|{'neg': 0.0, 'neu...|Australia|
|450|null| So Australia's t...|    DrStephR|Sydney, New South...|[('Australia', 'G...|{'neg': 0.296, 'n...|Australia|
|737|null|Luckily the non-s...|  mfullilove|            

In [25]:
TWITTER_oz_data =TWITTER_data.filter("country=='Australia'")

In [26]:
TWITTER_oz_data.explain()

== Physical Plan ==
*(1) Project [_c0#42, geo#43, text#44, user#45, location#46, entities#47, sentiment#48, country#49]
+- *(1) Filter (isnotnull(country#49) && (country#49 = Australia))
   +- *(1) FileScan csv [_c0#42,geo#43,text#44,user#45,location#46,entities#47,sentiment#48,country#49] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_dataset.csv], PartitionFilters: [], PushedFilters: [IsNotNull(country), EqualTo(country,Australia)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...


In [27]:
TWITTER_oz_data = TWITTER_data.filter("country=='Australia' and location like 'Syd%'")

In [28]:
TWITTER_oz_data.explain()

== Physical Plan ==
*(1) Project [_c0#42, geo#43, text#44, user#45, location#46, entities#47, sentiment#48, country#49]
+- *(1) Filter (((isnotnull(country#49) && isnotnull(location#46)) && (country#49 = Australia)) && StartsWith(location#46, Syd))
   +- *(1) FileScan csv [_c0#42,geo#43,text#44,user#45,location#46,entities#47,sentiment#48,country#49] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_dataset.csv], PartitionFilters: [], PushedFilters: [IsNotNull(country), IsNotNull(location), EqualTo(country,Australia), StringStartsWith(location,S..., ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...


In [29]:
TWITTER_oz_data = TWITTER_data.filter(col("location").startswith("S"))

In [30]:
TWITTER_oz_data.show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+---------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|  country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+---------+
|  9|null|How they’re deali...|  _______coolio| South East, England|['', ('Portsmouth...|{'neg': 0.126, 'n...|       UK|
| 33|null|" #Coronavirus wa...| glass_jonathan|Some-Where in Canada|[('Italy', 'GPE')...|{'neg': 0.107, 'n...|   Canada|
| 34|null| There is no obje...|       SFAuGrrl|   San Francisco, CA|['', ('FamiliesFi...|{'neg': 0.224, 'n...|      USA|
| 46|null| TLDR explains. #...|   mamandurria1|   Sao Paulo, Brazil|                ['']|{'neg': 0.0, 'neu...|   Brazil|
| 75|null| A woman in her 6...|  SusanTaylorSD|          San Diego |[('60s', 'DATE'),...|{'neg': 0.251, 'n...|      USA|
| 77|null| There is no obje...| 

In [31]:
TWITTER_oz_data.explain()

== Physical Plan ==
*(1) Project [_c0#42, geo#43, text#44, user#45, location#46, entities#47, sentiment#48, country#49]
+- *(1) Filter (isnotnull(location#46) && StartsWith(location#46, S))
   +- *(1) FileScan csv [_c0#42,geo#43,text#44,user#45,location#46,entities#47,sentiment#48,country#49] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_dataset.csv], PartitionFilters: [], PushedFilters: [IsNotNull(location), StringStartsWith(location,S)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...


In [32]:
TWITTER_data.take(5)

[Row(_c0=0, geo=None, text='What is God saying to us about #coronavirus ?', user='petodinice', location='Lagos', entities="[('about #', 'CARDINAL')]", sentiment="{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}", country='Nigeria'),
 Row(_c0=1, geo=None, text='"BREAKING: ""this is disappointing', user=' but i took the test"". Arsenal\'s Mikel Arteta tests positive for #coronavirus', location='  -… "', entities='JerryfranksonJF', sentiment='Abuja, Nigeria', country='"[(""Arsenal\'s Mikel Arteta""'),
 Row(_c0=2, geo=None, text=' #Coronavirus testing must be made free to the public if we are going to understand the scope of this crisis. Anything le…', user='cek422', location='Pennsylvania, USA', entities='[]', sentiment="{'neg': 0.173, 'neu': 0.71, 'pos': 0.117, 'compound': -0.3767}", country='USA'),
 Row(_c0=3, geo=None, text=' Get ready for mass event crowd cancellations across the World starting this weekend: cricket in #Australia in empty st…', user='InfectiousDz', locatio

In [33]:
TWITTER_data.show(5)

+---+----+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+
|_c0| geo|                text|                user|         location|            entities|           sentiment|             country|
+---+----+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+
|  0|null|What is God sayin...|          petodinice|            Lagos|[('about #', 'CAR...|{'neg': 0.0, 'neu...|             Nigeria|
|  1|null|"BREAKING: ""this...| but i took the t...|             -… "|     JerryfranksonJF|      Abuja, Nigeria|"[(""Arsenal's Mi...|
|  2|null| #Coronavirus tes...|              cek422|Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|                 USA|
|  3|null| Get ready for ma...|        InfectiousDz|              NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|                 USA|
|  4|null| The #coronavirus...|          vic_gibson|          

In [34]:
x= spark.sparkContext.parallelize([5,10,15])
y = x.map(lambda x:(x,x*2))
print(x.collect())
print(y.collect())

[5, 10, 15]
[(5, 10), (10, 20), (15, 30)]


In [35]:
x= spark.sparkContext.parallelize([5,10,15])
y = x.flatMap(lambda x:(x,x*2))
print(x.collect())
print(y.collect())

[5, 10, 15]
[5, 10, 10, 20, 15, 30]


In [36]:
TWITTER_data.select("user","sentiment").show()

+--------------------+--------------------+
|                user|           sentiment|
+--------------------+--------------------+
|          petodinice|{'neg': 0.0, 'neu...|
| but i took the t...|      Abuja, Nigeria|
|              cek422|{'neg': 0.173, 'n...|
|        InfectiousDz|{'neg': 0.085, 'n...|
|          vic_gibson|{'neg': 0.0, 'neu...|
|      StewartNgilana|{'neg': 0.178, 'n...|
|           BWheatnyc|{'neg': 0.098, 'n...|
|             LorseaR|{'neg': 0.123, 'n...|
|         straightj23|{'neg': 0.0, 'neu...|
|       _______coolio|{'neg': 0.126, 'n...|
|      champagneaylin|{'neg': 0.221, 'n...|
|       YorkLawLondon|{'neg': 0.0, 'neu...|
|      BeesonMargaret|{'neg': 0.227, 'n...|
|      harrytiffanyiv|{'neg': 0.0, 'neu...|
|         grammyheath|{'neg': 0.0, 'neu...|
|      barbara_ellena|{'neg': 0.0, 'neu...|
|               fatal|{'neg': 0.139, 'n...|
|          pwjkmiller|{'neg': 0.218, 'n...|
|        Mrrandy123RP|{'neg': 0.0, 'neu...|
|           aliensky0|{'neg': 0.

In [37]:
TWITTER_data.rdd.map(lambda new: new.text.split(" ")).take(10)

[['What', 'is', 'God', 'saying', 'to', 'us', 'about', '#coronavirus', '?'],
 ['"BREAKING:', '""this', 'is', 'disappointing'],
 ['',
  '#Coronavirus',
  'testing',
  'must',
  'be',
  'made',
  'free',
  'to',
  'the',
  'public',
  'if',
  'we',
  'are',
  'going',
  'to',
  'understand',
  'the',
  'scope',
  'of',
  'this',
  'crisis.',
  'Anything',
  'le…'],
 ['',
  'Get',
  'ready',
  'for',
  'mass',
  'event',
  'crowd',
  'cancellations',
  'across',
  'the',
  'World',
  'starting',
  'this',
  'weekend:',
  'cricket',
  'in',
  '#Australia',
  'in',
  'empty',
  'st…'],
 ['',
  'The',
  '#coronavirus',
  'pandemic',
  'is',
  'revealing',
  'just',
  'how',
  'closely',
  'we',
  'are',
  'all',
  'bound',
  'together...[A',
  'thread]',
  ''],
 ['',
  'COVID-19',
  'update',
  'as',
  'of',
  'this',
  'morning:1.',
  'Death',
  'toll',
  'in',
  'Italy',
  'passes',
  '1,0002.',
  "Arsenal's",
  'head',
  'coach',
  'Arteta',
  'tests',
  'positive3.',
  'US…'],
 ['',
  'It

In [38]:
TWITTER_data.rdd.flatMap(lambda new: new.text.split(" ")).take(50)

['What',
 'is',
 'God',
 'saying',
 'to',
 'us',
 'about',
 '#coronavirus',
 '?',
 '"BREAKING:',
 '""this',
 'is',
 'disappointing',
 '',
 '#Coronavirus',
 'testing',
 'must',
 'be',
 'made',
 'free',
 'to',
 'the',
 'public',
 'if',
 'we',
 'are',
 'going',
 'to',
 'understand',
 'the',
 'scope',
 'of',
 'this',
 'crisis.',
 'Anything',
 'le…',
 '',
 'Get',
 'ready',
 'for',
 'mass',
 'event',
 'crowd',
 'cancellations',
 'across',
 'the',
 'World',
 'starting',
 'this',
 'weekend:']

In [39]:
CV_data.show(10)

+---+----------------+---------+--------+---------+-------------------+---------+-----+---------+----------------+----+
|_c0|           State|  Country|     Lat|     Long|               Date|Confirmed|Death|Recovered|   state_cleaned|City|
+---+----------------+---------+--------+---------+-------------------+---------+-----+---------+----------------+----+
|  0|            null| Thailand|    15.0|    101.0|2020-01-22 00:00:00|      2.0|  0.0|      0.0|         Bangkok|null|
|  1|            null|    Japan|    36.0|    138.0|2020-01-22 00:00:00|      2.0|  0.0|      0.0|         Hiraide|null|
|  2|            null|Singapore|  1.2833| 103.8333|2020-01-22 00:00:00|      0.0|  0.0|      0.0|       Singapore|null|
|  3|            null|    Nepal| 28.1667|    84.25|2020-01-22 00:00:00|      0.0|  0.0|      0.0|       Kathmandu|null|
|  4|            null| Malaysia|     2.5|    112.5|2020-01-22 00:00:00|      0.0|  0.0|      0.0|         Sarawak|null|
|  5|British Columbia|   Canada| 49.2827

In [40]:
CV_data.filter("Country='Australia'").show(100)

+----+--------------------+---------+--------+--------+-------------------+---------+-----+---------+--------------------+----+
| _c0|               State|  Country|     Lat|    Long|               Date|Confirmed|Death|Recovered|       state_cleaned|City|
+----+--------------------+---------+--------+--------+-------------------+---------+-----+---------+--------------------+----+
|   6|     New South Wales|Australia|-33.8688|151.2093|2020-01-22 00:00:00|      0.0|  0.0|      0.0|     New South Wales|null|
|   7|            Victoria|Australia|-37.8136|144.9631|2020-01-22 00:00:00|      0.0|  0.0|      0.0|            Victoria|null|
|   8|          Queensland|Australia|-28.0167|   153.4|2020-01-22 00:00:00|      0.0|  0.0|      0.0|          Queensland|null|
|  19|     South Australia|Australia|-34.9285|138.6007|2020-01-22 00:00:00|      0.0|  0.0|      0.0|     South Australia|null|
|  22|From Diamond Prin...|Australia| 35.4437| 139.638|2020-01-22 00:00:00|      0.0|  0.0|      0.0|Fro

In [41]:
CV_data.filter("Country='Australia'").sort(col('Date'),ascending=False).show()

+-----+--------------------+---------+--------+--------+-------------------+---------+-----+---------+--------------------+----+
|  _c0|               State|  Country|     Lat|    Long|               Date|Confirmed|Death|Recovered|       state_cleaned|City|
+-----+--------------------+---------+--------+--------+-------------------+---------+-----+---------+--------------------+----+
|26607|            Victoria|Australia|-37.8136|144.9631|2020-03-18 00:00:00|    121.0|  0.0|      8.0|            Victoria|null|
|26649|   Western Australia|Australia|-31.9505|115.8605|2020-03-18 00:00:00|     35.0|  1.0|      0.0|   Western Australia|null|
|26608|          Queensland|Australia|-28.0167|   153.4|2020-03-18 00:00:00|     94.0|  0.0|      8.0|          Queensland|null|
|26661|            Tasmania|Australia|-41.4545|145.9707|2020-03-18 00:00:00|     10.0|  0.0|      0.0|            Tasmania|null|
|26671|  Northern Territory|Australia|-12.4634|130.8456|2020-03-18 00:00:00|      1.0|  0.0|     

In [42]:
CV_data.filter("Country='Australia'").orderBy(col('Date'),ascending=False).show()

+-----+--------------------+---------+--------+--------+-------------------+---------+-----+---------+--------------------+----+
|  _c0|               State|  Country|     Lat|    Long|               Date|Confirmed|Death|Recovered|       state_cleaned|City|
+-----+--------------------+---------+--------+--------+-------------------+---------+-----+---------+--------------------+----+
|26607|            Victoria|Australia|-37.8136|144.9631|2020-03-18 00:00:00|    121.0|  0.0|      8.0|            Victoria|null|
|26649|   Western Australia|Australia|-31.9505|115.8605|2020-03-18 00:00:00|     35.0|  1.0|      0.0|   Western Australia|null|
|26608|          Queensland|Australia|-28.0167|   153.4|2020-03-18 00:00:00|     94.0|  0.0|      8.0|          Queensland|null|
|26661|            Tasmania|Australia|-41.4545|145.9707|2020-03-18 00:00:00|     10.0|  0.0|      0.0|            Tasmania|null|
|26671|  Northern Territory|Australia|-12.4634|130.8456|2020-03-18 00:00:00|      1.0|  0.0|     

In [43]:
CV_data.filter("Country='Australia'").sortWithinPartitions(col('Date'),col('Confirmed'),ascending=False).show()

+-----+--------------------+---------+--------+--------+-------------------+---------+-----+---------+--------------------+----+
|  _c0|               State|  Country|     Lat|    Long|               Date|Confirmed|Death|Recovered|       state_cleaned|City|
+-----+--------------------+---------+--------+--------+-------------------+---------+-----+---------+--------------------+----+
|26606|     New South Wales|Australia|-33.8688|151.2093|2020-03-18 00:00:00|    267.0|  5.0|      4.0|     New South Wales|null|
|26607|            Victoria|Australia|-37.8136|144.9631|2020-03-18 00:00:00|    121.0|  0.0|      8.0|            Victoria|null|
|26608|          Queensland|Australia|-28.0167|   153.4|2020-03-18 00:00:00|     94.0|  0.0|      8.0|          Queensland|null|
|26619|     South Australia|Australia|-34.9285|138.6007|2020-03-18 00:00:00|     37.0|  0.0|      3.0|     South Australia|null|
|26649|   Western Australia|Australia|-31.9505|115.8605|2020-03-18 00:00:00|     35.0|  1.0|     

In [44]:
CV_data.describe().show()

+-------+-----------------+---------+-----------+------------------+-------------------+------------------+------------------+------------------+-------------+-----------+
|summary|              _c0|    State|    Country|               Lat|               Long|         Confirmed|             Death|         Recovered|state_cleaned|       City|
+-------+-----------------+---------+-----------+------------------+-------------------+------------------+------------------+------------------+-------------+-----------+
|  count|            27075|    18297|      27075|             27075|              27075|             26884|             26884|             26884|        27075|      14193|
|   mean|          13537.0|     null|       null|31.287152842104447|-35.511753263158006|150.31249070078857|  4.96436542181223| 56.59284332688588|         null|       null|
| stddev|7816.023605389124|     null|       null|19.032233293159134|  79.82706281478735|2462.0157547275367|102.60625186423131|1281.794385056

In [45]:
CV_data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Confirmed: double (nullable = true)
 |-- Death: double (nullable = true)
 |-- Recovered: double (nullable = true)
 |-- state_cleaned: string (nullable = true)
 |-- City: string (nullable = true)



In [46]:
CV_data.filter('Confirmed>20000').sort(col('Confirmed'),ascending=False).show()

+-----+-----+-------+-------+--------+-------------------+---------+------+---------+-------------+----+
|  _c0|State|Country|    Lat|    Long|               Date|Confirmed| Death|Recovered|state_cleaned|City|
+-----+-----+-------+-------+--------+-------------------+---------+------+---------+-------------+----+
|26754|Hubei|  China|30.9756|112.2707|2020-03-18 00:00:00|  67800.0|3122.0|  56927.0|        Hubei|null|
|26279|Hubei|  China|30.9756|112.2707|2020-03-17 00:00:00|  67799.0|3111.0|  56003.0|        Hubei|null|
|25804|Hubei|  China|30.9756|112.2707|2020-03-16 00:00:00|  67798.0|3099.0|  55142.0|        Hubei|null|
|25329|Hubei|  China|30.9756|112.2707|2020-03-15 00:00:00|  67794.0|3085.0|  54288.0|        Hubei|null|
|24854|Hubei|  China|30.9756|112.2707|2020-03-14 00:00:00|  67790.0|3075.0|  52960.0|        Hubei|null|
|24379|Hubei|  China|30.9756|112.2707|2020-03-13 00:00:00|  67786.0|3062.0|  51553.0|        Hubei|null|
|23904|Hubei|  China|30.9756|112.2707|2020-03-12 00:00:

In [47]:
CV_data.filter('Confirmed>20000').approxQuantile('Confirmed',[0.25,0.5,0.75,1.0],0.95)

[21157.0, 21157.0, 21157.0, 67800.0]

In [48]:
CV_max_data = CV_data.agg({'Date':"max", 'confirmed':"max"})

In [49]:
CV_max_data.show()

+--------------+-------------------+
|max(confirmed)|          max(Date)|
+--------------+-------------------+
|       67800.0|2020-03-18 00:00:00|
+--------------+-------------------+



In [50]:
import pyspark.sql.functions as SQL

In [51]:
CV_data.groupby("Country","state_cleaned").agg(SQL.max("Date")).show()

+--------------+--------------------+-------------------+
|       Country|       state_cleaned|          max(Date)|
+--------------+--------------------+-------------------+
|      Cameroon|             Yaounde|2020-03-18 00:00:00|
|        Cyprus|             Nicosia|2020-03-18 00:00:00|
|            US|            Michigan|2020-03-18 00:00:00|
|         China|             Qinghai|2020-03-18 00:00:00|
|      Portugal|              Lisbon|2020-03-18 00:00:00|
|            US|            Colorado|2020-03-18 00:00:00|
|United Kingdom|      Cayman Islands|2020-03-18 00:00:00|
|            US|            Missouri|2020-03-18 00:00:00|
|         China|              Hainan|2020-03-18 00:00:00|
|     Australia|Australian Capita...|2020-03-18 00:00:00|
|            US|                Guam|2020-03-18 00:00:00|
|        France|             Reunion|2020-03-18 00:00:00|
|      Colombia|        Cundinamarca|2020-03-18 00:00:00|
|          Cuba|              Havana|2020-03-18 00:00:00|
|     Mauritiu

In [52]:
CV_data.join(CV_data.groupby("Country","state_cleaned").agg(SQL.max("Date").alias("Date")), on=['Country','state_cleaned','Date'],how='inner').sort("Confirmed",ascending=False).show()

+--------------+----------------+-------------------+-----+--------------+-------+-------------------+---------+------+---------+--------+
|       Country|   state_cleaned|               Date|  _c0|         State|    Lat|               Long|Confirmed| Death|Recovered|    City|
+--------------+----------------+-------------------+-----+--------------+-------+-------------------+---------+------+---------+--------+
|         China|           Hubei|2020-03-18 00:00:00|26754|         Hubei|30.9756|           112.2707|  67800.0|3122.0|  56927.0|    null|
|         Italy|            Rome|2020-03-18 00:00:00|26616|          null|   43.0|               12.0|  35713.0|2978.0|   4025.0|    null|
|          Iran|          Tehran|2020-03-18 00:00:00|26755|          null|   32.0|               53.0|  17361.0|1135.0|   5389.0|    null|
|         Spain|          Toledo|2020-03-18 00:00:00|26618|          null|   40.0|               -4.0|  13910.0| 623.0|   1081.0|    null|
|       Germany|          B

In [53]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

In [54]:
window = Window().partitionBy("Country","state_cleaned").orderBy(col("Date").desc())

In [55]:
CV_data.withColumn("row_num",row_number().over(window)).show()

+-----+-----+--------+-----+-------+-------------------+---------+-----+---------+-------------+----+-------+
|  _c0|State| Country|  Lat|   Long|               Date|Confirmed|Death|Recovered|state_cleaned|City|row_num|
+-----+-----+--------+-----+-------+-------------------+---------+-----+---------+-------------+----+-------+
|26679| null|Cameroon|3.848|11.5021|2020-03-18 00:00:00|     10.0|  0.0|      0.0|      Yaounde|null|      1|
|26204| null|Cameroon|3.848|11.5021|2020-03-17 00:00:00|     10.0|  0.0|      0.0|      Yaounde|null|      2|
|25729| null|Cameroon|3.848|11.5021|2020-03-16 00:00:00|      4.0|  0.0|      0.0|      Yaounde|null|      3|
|25254| null|Cameroon|3.848|11.5021|2020-03-15 00:00:00|      2.0|  0.0|      0.0|      Yaounde|null|      4|
|24779| null|Cameroon|3.848|11.5021|2020-03-14 00:00:00|      2.0|  0.0|      0.0|      Yaounde|null|      5|
|24304| null|Cameroon|3.848|11.5021|2020-03-13 00:00:00|      2.0|  0.0|      0.0|      Yaounde|null|      6|
|23829| nu

In [56]:
CV_data.groupBy('Country').pivot("Date").agg(SQL.sum("confirmed")).show()

+-----------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------

In [57]:
CV_data.filter("Country=='Australia'").crosstab("State","Date").show()

+--------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+----------

In [58]:
CV_data.groupby("Country").agg({"Confirmed":'sum',"Death":'sum',"Recovered":'sum'}).orderBy("sum(Death)",ascending=False).show()

+--------------+--------------+----------+--------------+
|       Country|sum(Recovered)|sum(Death)|sum(Confirmed)|
+--------------+--------------+----------+--------------+
|         China|     1428378.0|  103112.0|     3124622.0|
|         Italy|       21954.0|   16221.0|      238049.0|
|          Iran|       43977.0|    7166.0|      150833.0|
|         Spain|        4398.0|    2325.0|       64917.0|
|  Korea, South|        7443.0|    1048.0|      137889.0|
|        France|         318.0|     930.0|       48132.0|
|            US|         355.0|     691.0|       34228.0|
|         Japan|        1938.0|     275.0|       12584.0|
|United Kingdom|         473.0|     268.0|       12086.0|
|   Netherlands|          10.0|     189.0|       10548.0|
|   Cruise Ship|        2894.0|     151.0|       22668.0|
|   Switzerland|          66.0|     125.0|       15979.0|
|   Philippines|          51.0|     108.0|        1118.0|
|          Iraq|         213.0|     107.0|        1377.0|
|       German

In [59]:
CV_data.withColumn("Okay_now",CV_data.Confirmed - CV_data.Recovered - CV_data.Death).sort("Okay_now",ascending=False).show()

+-----+-----+-------+-------+--------+-------------------+---------+------+---------+-------------+----+--------+
|  _c0|State|Country|    Lat|    Long|               Date|Confirmed| Death|Recovered|state_cleaned|City|Okay_now|
+-----+-----+-------+-------+--------+-------------------+---------+------+---------+-------------+----+--------+
|12979|Hubei|  China|30.9756|112.2707|2020-02-18 00:00:00|  61682.0|1921.0|   9128.0|        Hubei|null| 50633.0|
|12504|Hubei|  China|30.9756|112.2707|2020-02-17 00:00:00|  59989.0|1789.0|   7862.0|        Hubei|null| 50338.0|
|12029|Hubei|  China|30.9756|112.2707|2020-02-16 00:00:00|  58182.0|1696.0|   6639.0|        Hubei|null| 49847.0|
|13454|Hubei|  China|30.9756|112.2707|2020-02-19 00:00:00|  62031.0|2029.0|  10337.0|        Hubei|null| 49665.0|
|11554|Hubei|  China|30.9756|112.2707|2020-02-15 00:00:00|  56249.0|1596.0|   5623.0|        Hubei|null| 49030.0|
|14404|Hubei|  China|30.9756|112.2707|2020-02-21 00:00:00|  62662.0|2144.0|  11881.0|   

In [60]:
CV_data_new = CV_data.withColumn("Okay_now",CV_data.Confirmed - CV_data.Recovered - CV_data.Death)

In [61]:
CV_data_new.groupBy("Country").sum("Okay_now").orderBy("sum(Okay_now)",ascending=False).show()

+--------------+-------------+
|       Country|sum(Okay_now)|
+--------------+-------------+
|         China|    1593132.0|
|         Italy|     199874.0|
|  Korea, South|     129398.0|
|          Iran|      99690.0|
|         Spain|      58194.0|
|       Germany|      52971.0|
|        France|      46884.0|
|            US|      33182.0|
|   Cruise Ship|      19623.0|
|   Switzerland|      15788.0|
|United Kingdom|      11345.0|
|         Japan|      10371.0|
|   Netherlands|      10349.0|
|        Norway|      10203.0|
|        Sweden|       8728.0|
|       Belgium|       7548.0|
|       Austria|       7194.0|
|       Denmark|       7102.0|
|      Malaysia|       3710.0|
|         Qatar|       2995.0|
+--------------+-------------+
only showing top 20 rows

